# Geometric Brownian Motion

In [ ]:
using KadanoffBaym
using LinearAlgebra

In [ ]:
function solution(S_0::Float64, σ::Float64)
    F_ana(t1, t2) = S_0^2 * exp(mu * (t1 + t2)) * (exp(σ^2 * min(t1, t2)) - 1);
    
    f_vert(_, _, t1, t2) = [0., mu * F[t1, t2]]
    f_diag(_, times, t)  = [mu * S[t, t], 2mu * F[t, t] + σ^2 * (S[t, t]^2 + F[t, t])]
  
    S = GreenFunction(S_0 * ones(1,1), SymmetricSymmetry)
    F = GreenFunction(0.0 * ones(1,1), SymmetricSymmetry)
    
    sol = kbsolve(f_vert, f_diag, [S, F], (0.0, T), atol=1e-8, rtol=1e-6, dtini=1e-10)
         
    return [sol.t, S.data, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
T = 1.0
mu = 1.
t_scale = (mu == 0. ? 1. : abs(mu))

S_0 = [1., 2., 5.]
sigma = [1., .5, .1]

s = mapreduce(solution, hcat, S_0, sigma);

## Plotting

In [ ]:
using PyPlot

font_size = 16

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=font_size)
PyPlot.matplotlib.rc("axes", labelsize=font_size)
PyPlot.matplotlib.rc("xtick.major", size=8)
PyPlot.matplotlib.rc("ytick.major", size=8)
PyPlot.matplotlib.rc("xtick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("ytick.minor", visible=true, size=4)
PyPlot.matplotlib.rc("xtick", top=true, direction="inout")
PyPlot.matplotlib.rc("ytick", right=true, direction="inout")

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(sigma)
    if k == 1
        plot(s[1,k], diag(s[3,k]), label="\$F(t, t)\$", lw=1.5, c=colors[k], ls=lss[k])
        plot(s[1,k], diag(s[2,k]), label="\$\\bar{S}(t)\$", lw=3.5, c=colors[k], ls=lss[k])
    else
        plot(s[1,k], diag(s[3,k]), lw=1.5, c=colors[k], ls=lss[k])
        plot(s[1,k], diag(s[2,k]), lw=3.5, c=colors[k], ls=lss[k])
    end
end
xlim(0, t_scale * T)
ylim(0, 15)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\mu t\$")
# ylabel("\$F(t, t)\$")
ax.legend(loc="best", handlelength=1.5, frameon=false, borderpad=0, labelspacing=0.25)
leg = ax.get_legend()
leg.legendHandles[1].set_color("k")
leg.legendHandles[2].set_color("k")

ax = subplot(122)
plot([], [], label="\$\\sigma/\\mu\$", c="w")
for k in eachindex(sigma)
    semilogy(s[1,k], abs.(diag(s[3,k])  .- diag(s[4,k])), lw=1.5, c=colors[k], label="\$"*string(sigma[k]/(mu))*"\$", ls=lss[k])
end

xlim(0, t_scale * T)
ylim(1e-9, 1e-5)
# yticks([0, 1, 2, 3, 4, 5])
xlabel("\$\\mu t\$")
ylabel("\$\\left|F(t, t) - \\mathcal{F}(t, t)\\right|\$", labelpad=16)
ax.yaxis.set_label_position("right")
ax.legend(loc="best", handlelength=1.9, frameon=false, borderpad=0, labelspacing=0.25)
# ticklabel_format(axis="y", style="sci", scilimits=(-0, 0))

tight_layout(pad=0.25, w_pad=1, h_pad=0)

savefig("geometric_brownian_motion_example_T.pdf")

## Testing

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
Y, X = meshgrid(s[1,1], s[1,1]);

In [ ]:
figure(figsize=(7, 3))

vmin = 0
# vmax = D/(2lambda)
ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, s[3,1], cmap=cmap, rasterized=true)#, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, (abs.(s[3,1] .- s[4,1])), cmap="gist_gray", rasterized=true)#, vmin=1e-9, vmax=1e-6)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
# ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
# ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)
# savefig("brownian_motion_example_t_tp.pdf")